In [1]:
from ten_con import *
from pauli_utils import *

In [2]:
from copy import copy
import numpy as np
from numba import jit

In [3]:
@jit
def mat_compose(coef_matrix:np.matrix):
    mat = coef_matrix
    _2n = mat.shape[0] # 2^n
    steps = int(np.log2(_2n))# n
    unit_size= 1
 
    for step in range(steps):
        step1 = step+1
        mat_size = int(2*(unit_size))
        indexes = np.arange(_2n/(2**step1)).astype(np.uint)
        indexes_ij = mat_size * indexes
        for i in indexes_ij:
            for j in indexes_ij:
                # (i, j)
                r1i     = i
                r1f2i   = r1i + unit_size
                c1i     = j
                c1f2i   = c1i + +unit_size
                r2f     = r1f2i + unit_size
                c2f     = c1f2i + unit_size
                # I - Z
                coef = 1
                mat[r1i: r1f2i, c1i:c1f2i] = mat[r1i: r1f2i, c1i:c1f2i] + coef*mat[r1f2i: r2f, c1f2i:c2f]
                mat[r1f2i: r2f, c1f2i:c2f] = mat[r1i: r1f2i, c1i:c1f2i] -2*coef *mat[r1f2i: r2f, c1f2i:c2f]
                # X -Y
                coef = -1j
                mat[r1i: r1f2i, c1f2i:c2f] = mat[r1i: r1f2i, c1f2i:c2f]  + coef*mat[r1f2i: r2f, c1i:c1f2i]
                mat[r1f2i: r2f, c1i:c1f2i] = mat[r1i: r1f2i, c1f2i:c2f] - 2*coef*mat[r1f2i: r2f, c1i:c1f2i]
                # Here is a but with numba jit, the below two line code is correct
                # However, with jit decorator the phase get different. 
                #mat[r1i: r1f2i, c1f2i:c2f] += coef*mat[r1f2i: r2f, c1i:c1f2i]
                #mat[r1f2i: r2f, c1i:c1f2i] = mat[r1i: r1f2i, c1f2i:c2f] - 2*coef*mat[r1f2i: r2f, c1i:c1f2i]
        
        unit_size *=2
    return mat

In [4]:
H = pstr2mat("XX") + 2j*pstr2mat("ZI") + pstr2mat("XY") + pstr2mat("ZX")

In [5]:
H

matrix([[ 0.+2.j,  1.+0.j,  0.+0.j,  1.-1.j],
        [ 1.+0.j,  0.+2.j,  1.+1.j,  0.+0.j],
        [ 0.+0.j,  1.-1.j,  0.-2.j, -1.+0.j],
        [ 1.+1.j,  0.+0.j, -1.+0.j,  0.-2.j]])

In [6]:
coef = copy(H)
mat_decompose(coef)

matrix([[0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+2.j, 1.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [7]:
restore = copy(coef)
mat_compose(restore)

matrix([[ 0.+2.j,  1.+0.j,  0.+0.j,  1.-1.j],
        [ 1.+0.j,  0.+2.j,  1.+1.j,  0.+0.j],
        [ 0.+0.j,  1.-1.j,  0.-2.j, -1.+0.j],
        [ 1.+1.j,  0.+0.j, -1.+0.j,  0.-2.j]])

In [8]:
restore = copy(coef)
mat_compose_no_jit(restore)

matrix([[ 0.+2.j,  1.+0.j,  0.+0.j,  1.-1.j],
        [ 1.+0.j,  0.+2.j,  1.+1.j,  0.+0.j],
        [ 0.+0.j,  1.-1.j,  0.-2.j, -1.+0.j],
        [ 1.+1.j,  0.+0.j, -1.+0.j,  0.-2.j]])

## Numpy

In [2]:
import numpy as np

In [3]:
arr = np.array(
    [
    [0., 1., 0.2, 3.2, 0.1, 0.00001],
    [0., 1., 0.2, 3.2, 0.1, 0.00001],
    [0., 1., 0.2, 3.2, 0.1, 0.00001],
    [0., 1., 0.2, 3.2, 0.1, 0.00001]
    ]
)

In [7]:
np.nonzero(arr>0.1)

(array([0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3], dtype=int64),
 array([1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3], dtype=int64))